In [1]:
from trainlib.FileCollection import FileCollection
from trainlib.config import Config
import trainlib.cuts as cuts
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Welcome to JupyROOT 6.10/09


In [2]:
import xgboost as xgb

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
#candidate_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "leading_jet_eta", "leading_jet_pt", "leading_jet_sin_phi", "leading_jet_cos_phi", "leading_lep_pt", "leading_lep_eta", "leading_lep_sin_phi", "leading_lep_cos_phi", "leading_extra_lep_pt", "leading_extra_lep_eta", "leading_extra_lep_sin_phi", "leading_extra_lep_cos_phi", "nCleanedJets", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav", "costhetastar", "helphi", "helcosthetaZ1", "helcosthetaZ2", "phistarZ1", "phistarZ2", "xi", "xistar"]
candidate_branches = ["nCleanedJets", "PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav", "costhetastar", "helphi", "helcosthetaZ1", "helcosthetaZ2", "phistarZ1", "phistarZ2", "xi", "xistar"]
MELA_branches = ["D_VBF2j_ggH_ME", "D_VBF1j_ggH_ME", "D_WHh_ggH_ME", "D_ZHh_ggH_ME", "D_WHh_ZHh_ME", "D_VBF2j_WHh_ME", "D_VBF2j_ZHh_ME"]
list_branches = ["Jet", "Lep", "ExtraLep"]
pt_limits = [30.0, 0.0, 0.0]

In [36]:
allbranches = ["JetPt", "JetEta", "JetPhi", "LepPt", "LepEta", "LepPhi", "ExtraLepPt", "ExtraLepEta", "ExtraLepPhi"] + candidate_branches + MELA_branches + ["LHEAssociatedParticleId", "GenAssocLep1Id", "GenAssocLep2Id", "training_weight"]

In [37]:
allinputbranches = candidate_branches + MELA_branches

In [38]:
#MC_path = "/data_CMS/cms/wind/CJLST_NTuples_randomizeda/"
MC_path = "/data_CMS/cms/wind/CJLST_NTuples/"

In [39]:
def WHhadr0j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] == 0

def WHhadr01j_cut(row):
    return cuts.WHhadr_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def WHhadr1j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] == 1

def WHhadr2j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] >= 2

def ZHhadr0j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] == 0

def ZHhadr01j_cut(row):
    return cuts.ZHhadr_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def ZHhadr1j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] == 1

def ZHhadr2j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] >= 2

def mZZ0j_cut(row):
    return cuts.mZZ_cut(row) and row["nCleanedJetsPt30"] == 0

def mZZ01j_cut(row):
    return cuts.mZZ_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def mZZ1j_cut(row):
    return cuts.mZZ_cut(row) and row["nCleanedJetsPt30"] == 1

def mZZ2j_cut(row):
    return cuts.mZZ_cut(row) and row["nCleanedJetsPt30"] >= 2

In [40]:
# all the model combinations for which neural networks are currently trained
discriminant_pairs = [("VBF2j", "ggH2j"), ("VBF1j", "ggH1j"), ("VBF0j", "ggH0j"), ("WHh2j", "ggH2j"), 
                     ("WHh1j", "ggH1j"), ("WHh0j", "ggH0j"), ("ZHh2j", "ggH2j"), ("ZHh1j", "ggH1j"), 
                      ("ZHh0j", "ggH0j"), ("WHh2j", "ZHh2j"), ("WHh01j", "ZHh01j"), ("VBF2j", "WHh2j"),
                     ("VBF1j", "WHh1j"), ("VBF0j", "WHh0j"), ("VBF2j", "ZHh2j"), ("VBF1j", "ZHh1j"), 
                      ("VBF0j", "ZHh0h"), ("WHl", "ggH"), ("WHl", "VBF"), ("WHl", "WHh"), ("WHl", "ZHh"),
                     ("WHl", "ZHl"), ("WHl", "ZHMET"), ("WHl", "ttHh"), ("WHl", "ttHl"), ("ZHh", "ZHl"),
                     ("ZHh", "ZHMET"), ("ZHh", "ttHh"), ("ZHh", "ttHl"), ("ZHl", "ggH"), ("ZHl", "VBF"),
                     ("ZHl", "WHh"), ("ZHl", "ZHMET"), ("ZHl", "ttHh"), ("ZHl", "ttHl"), ("ZHMET", "ggH"),
                     ("ZHMET", "VBF"), ("ZHMET", "WHh"), ("ZHMET", "ttHh"), ("ZHMET", "ttHl"), ("ttHh", "ggH"),
                      ("ttHh", "VBF"), ("ttHh", "WHh"), ("ttHh", "ttHl"), ("ttHl", "ggH"), ("ttHl", "VBF"),
                     ("ttHl", "WHh")]

In [41]:
def get_list_element(row, column_name, list_index):
    if list_index < len(row[column_name]):
        return row[column_name][list_index]
    else:
        return 0.0

In [42]:
def get_sorted_list_element(row, list_index, sorted_column, aux_columns, min_pt):
    sorted_objects = np.flipud(np.argsort(row[sorted_column]))
        
    if list_index < len(row[sorted_column]) and row[sorted_column][sorted_objects[list_index]] > min_pt:
        processed_row = {column_name + "_" + str(list_index): [float(row[column_name][sorted_objects[list_index]])] for column_name in [sorted_column] + aux_columns}
    else:
        processed_row = {column_name + "_" + str(list_index): [0.0] for column_name in [sorted_column] + aux_columns}
        
    return processed_row

In [43]:
def get_sorted_list_element_df(inframe, list_index, sorted_column, aux_columns, pt_limit):
    outframe = pd.DataFrame()
    for row in inframe.iterrows():
        processed_row = get_sorted_list_element(row[1], list_index, sorted_column, aux_columns, pt_limit)
        outframe = outframe.append(pd.DataFrame(processed_row))
    outframe = outframe.reset_index(drop = True)
    
    return outframe

In [44]:
def get_data(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits):
    print read_branches
    H1_df = H1_coll.get_data(read_branches, 0.0, 1.0)
    H0_df = H0_coll.get_data(read_branches, 0.0, 1.0)
    
    list_branches_unrolled = []
    
    H1_list_df = pd.DataFrame()
    H0_list_df = pd.DataFrame()
    for list_branch, pt_limit in zip(list_branches, pt_limits):
        for order in range(4):
            H1_cur_df = get_sorted_list_element_df(H1_df, order, list_branch + "Pt", [list_branch + "Eta", list_branch + "Phi"], pt_limit)
            H0_cur_df = get_sorted_list_element_df(H0_df, order, list_branch + "Pt", [list_branch + "Eta", list_branch + "Phi"], pt_limit)

            H1_list_df = pd.concat([H1_list_df, H1_cur_df], axis = 1)
            H0_list_df = pd.concat([H0_list_df, H0_cur_df], axis = 1)
            
            for unrolled_branch in H1_cur_df.columns:
                list_branches_unrolled.append(unrolled_branch)
            
    H1_df = pd.concat([H1_df, H1_list_df], axis = 1)
    H0_df = pd.concat([H0_df, H0_list_df], axis = 1)        
    
    complete_input_branches = input_branches + list_branches_unrolled
        
    H1_df = H1_df[complete_input_branches]
    H0_df = H0_df[complete_input_branches]

    H1_data = H1_df.as_matrix()
    H0_data = H0_df.as_matrix()
    H1_target = np.ones(np.shape(H1_data)[0])
    H0_target = np.zeros(np.shape(H0_data)[0])
    
    target = np.concatenate([H1_target, H0_target])
    data = np.concatenate([H1_data, H0_data])
    
    dmatrix = xgb.DMatrix(data, label = target, feature_names = complete_input_branches)
    
    return dmatrix

In [47]:
def get_feature_importance_list(disc_pair, mandatory_branches, optional_branches, list_branches, pt_limits, params, max_num_rounds):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    # first assemble the list of branches that can serve as input: it will *always* contain the mandatory branches,
    # and *can* contain some of the optional branches, if the name of the categories allows it
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if ("0j" in H1_name or "0j" in H0_name) and ("0j" in optional_branch):
            input_branches.append(optional_branch)
            
        if ("1j" in H1_name or "1j" in H0_name) and ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
        # can use all MELA values for events with at least 2 jets
        if ("2j" in H1_name or "2j" in H0_name):
            input_branches.append(optional_branch)
            
    print "input variables: " + str(input_branches)
    
    collections = {"VBF2j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ2j_cut},
                "VBF1j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ1j_cut},
                "VBF0j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ0j_cut},
                "VBF01j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ01j_cut},
                "VBF": {MC_path + "VBFH125/ZZ4lAnalysis.root": cuts.mZZ_cut},
                "ggH2j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ2j_cut},
                "ggH1j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ1j_cut},
                "ggH0j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ0j_cut},
                "ggH01j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ01j_cut},
                "ggH" : {MC_path + "ggH125/ZZ4lAnalysis.root": cuts.mZZ_cut},
                "WHh2j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr2j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr2j_cut},
                "WHh1j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr1j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr1j_cut},
                "WHh0j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr0j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr0j_cut},
                "WHh": {MC_path + "WplusH125/ZZ4lAnalysis.root": cuts.WHhadr_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": cuts.WHhadr_cut},
                "WHh01j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr01j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr01j_cut},
                "WHl": {MC_path + "WplusH125/ZZ4lAnalysis.root": cuts.WHlept_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": cuts.WHlept_cut},
                "ZHh2j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr2j_cut},
                "ZHh1j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr1j_cut},
                "ZHh01j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr01j_cut},
                "ZHh0j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr0j_cut},
                "ZHh": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHhadr_cut},
                "ZHl": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHlept_cut},
                "ttHh": {MC_path + "ttH125/ZZ4lAnalysis.root": cuts.ttHhadr_cut},
                "ttHl": {MC_path + "ttH125/ZZ4lAnalysis.root": cuts.ttHlept_cut},
                "ZHMET": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHMET_cut}
              }
    
    # get the training data for the BDT ...
    H1_coll_train = FileCollection(collections[H1_name], 0.0, 0.5)
    H0_coll_train = FileCollection(collections[H0_name], 0.0, 0.5)
    
    dtrain = get_data(H1_coll_train, H0_coll_train, allbranches, input_branches, list_branches, pt_limits)
    
    # ... and the validation data as well
    H1_coll_val = FileCollection(collections[H1_name], 0.5, 1.0)
    H0_coll_val = FileCollection(collections[H0_name], 0.5, 1.0)
    dval = get_data(H1_coll_val, H0_coll_val, allbranches, input_branches, list_branches, pt_limits)
    
    evallist = [(dtrain, 'train'), (dval, 'eval')]
    
    bst = xgb.train(params, dtrain, num_round, evals = evallist, early_stopping_rounds = 10)
    
    imp = bst.get_fscore()
    
    return [(key, val) for key, val in sorted(imp.iteritems(), key = lambda x: x[1], reverse = True)] 

In [48]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
num_round = 1000
implist = get_feature_importance_list(("WHh0j", "VBF0j"), candidate_branches, MELA_branches, list_branches, pt_limits, param, num_round)

input variables: ['nCleanedJets', 'PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'ZZMass', 'nExtraZ', 'Z1Mass', 'Z2Mass', 'Z1Pt', 'Z2Pt', 'ZZMassErr', 'ZZPt', 'ZZEta', 'ZZPhi', 'Z1Flav', 'Z2Flav', 'costhetastar', 'helphi', 'helcosthetaZ1', 'helcosthetaZ2', 'phistarZ1', 'phistarZ2', 'xi', 'xistar']
skimming /data_CMS/cms/wind/CJLST_NTuples/WminusH125/ZZ4lAnalysis.root
skimming /data_CMS/cms/wind/CJLST_NTuples/WplusH125/ZZ4lAnalysis.root
collection set up: 2 files, 56636 entries in total, 28317 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used
['JetPt', 'JetEta', 'JetPhi', 'LepPt', 'LepEta', 'LepPhi', 'ExtraLepPt', 'ExtraLepEta', 'ExtraLepPhi', 'nCleanedJets', 'PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'ZZMass', 'nExtraZ', 'Z1Mass', 'Z2Mass', 'Z1Pt', 'Z2Pt', 'ZZMassErr', 'ZZPt', 'ZZEta', 'ZZPhi', 'Z1Flav', 'Z

In [49]:
implist

[('Z1Pt', 4),
 ('LepEta_0', 3),
 ('Z2Mass', 3),
 ('nCleanedJets', 3),
 ('LepPt_3', 2),
 ('LepEta_2', 2),
 ('ZZMassErr', 2),
 ('ZZEta', 2),
 ('LepPt_1', 1),
 ('LepPhi_2', 1),
 ('ZZPhi', 1),
 ('PFMET', 1),
 ('phistarZ1', 1),
 ('LepEta_1', 1),
 ('LepEta_3', 1),
 ('xistar', 1),
 ('ZZPt', 1),
 ('LepPhi_3', 1),
 ('ZZMass', 1),
 ('helcosthetaZ2', 1)]

In [ ]:
%%capture
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
num_round = 1000
implist = get_feature_importance_list(("ZHh0j", "VBF0j"), candidate_branches, MELA_branches, param, num_round)

In [ ]:
implist

In [ ]:
%%capture
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
num_round = 1000
implist = get_feature_importance_list(("ZHh1j", "ggH1j"), candidate_branches, MELA_branches, param, num_round)

In [ ]:
implist

In [ ]:
%%capture
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
num_round = 1000
implist = get_feature_importance_list(("WHh1j", "ggH1j"), candidate_branches, MELA_branches, param, num_round)

In [ ]:
implist